In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import gsu
import drops
import utils
import random

In [ ]:
def show_trs(trs, box, view1=90.0, view2=0.0):
    """
    Show trajectory.
    """

    plt.figure(figsize=(20, 20))
    ax = plt.axes(projection='3d')
    ax.set_xlim(box[0], box[3])
    ax.set_ylim(box[1], box[4])
    ax.set_zlim(box[2], box[5])
    ax.view_init(view1, view2)
    
    for tr in trs:
    
        xs = [p[0] for p in tr]
        ys = [p[1] for p in tr]
        zs = [p[2] for p in tr]
    
        # Data for a three-dimensional line
        ax.plot3D(xs, ys, zs, 'gray')

In [ ]:
g = gsu.Grid()
g.load('../grids/cyl_stall.dat')
box = g.box()
print('grid box = ', box)
air = drops.read_vel_field_from_file('../grids/cyl_air.dat')
air.print_info()

In [ ]:
d = 1.0e-4
dt = 1.0e-6
stall_thr = 1.0e-6
max_fly_steps = 200

In [ ]:
stall_ind = g.get_variable_index('Stall')

# trajectories
trs = []

# show all faces
for f in g.Faces:
    #trs.append([f.Nodes[0].P, f.Nodes[1].P, f.Nodes[2].P, f.Nodes[0].P])
    trs.append([f.get_center(), f.get_point_above(d)])
    
# fly from wet faces
for f in g.Faces:
    stall_value = f.Data[stall_ind - 3]
    if stall_value >= stall_thr:
        res = air.fly(f.get_point_above(d), dt, g, max_fly_steps)
        if res[0] == 'C':
            print('imp2 water from face {0} to face {1} (stall value = {2})'.format(f.GloId, res[1].GloId, stall_value))
        trs.append(res[2])

In [ ]:
# show
show_trs(trs, box, view1=120.0, view2=10.0)